# Viper

Instructions:
https://docs.google.com/document/d/1DnPmgUXiO4bcdREncTJojadD2mrKZrXMGhPilTJlCDQ/edit

Settings:https://docs.google.com/spreadsheets/d/104t7KyOxfQvZH318MKxT0m7hMRRPvDMFVLIMd2YUVco/edit#gid=1354838607

Log: https://docs.google.com/spreadsheets/d/1oKod0EKYfZmrS9MUtmftmEcv6WPESP5-qMJ6DBuJ4kU/edit#gid=0


In [1]:
#Run this code block when there are updates to one of the modules
import time
import sys
import builtins
import filepath
import infrastructure
import account_health
from datetime import date
import pandas as pd
from datetime import timedelta

import viper_main
del sys.modules["viper_main"]
import viper_main

import scheduling
del sys.modules["scheduling"]
import scheduling

import content_selection
del sys.modules["content_selection"]
import content_selection

import offer_selection
del sys.modules["offer_selection"]
import offer_selection

import find_viper_settings
del sys.modules["find_viper_settings"]
import find_viper_settings

def set_inputs():
    fvs = find_viper_settings.inputs(lexi=lexi_clean, cobra=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, offers=offers)    
    cs = content_selection.inputs(fvs=fvs, lexi=lexi_clean, cobra=cobra, offers=offers, emit=emit)
    os = offer_selection.inputs(fvs=fvs, lexi=lexi_clean, cobra=cobra, offers=offers, emit=emit)
#     sched = scheduling.inputs(fvs=fvs, cs=cs, offs=os, cobra=cobra, cobra_clean=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, emit=emit, offers=offers, lexi=lexi, sc_dppub_affiliate=sc_dppub_affiliate, days_ahead=days_ahead)
    result = fvs, cs, os
    
def schedule_all_affiliates(affiliates):
    all_switch_notes = []
    cobra = cobra_clean.copy()
    #log_temp = fvs.load_log_old()
    log = fvs.load_log()
#     scheduled_offers_array = []
    for sc_dppub_affiliate in affiliates:
        print("\n\n\nBeginning Scheduling for {}:\n\n".format(sc_dppub_affiliate))
        sched = scheduling.inputs(fvs=fvs, cs=cs, offs=os, cobra=cobra, cobra_clean=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, emit=emit, offers=offers, lexi=lexi_clean, sc_dppub_affiliate=sc_dppub_affiliate, days_ahead=days_ahead, all_aff_log=log)
        #sched.format_existing_drops(sc_dppub_affiliate,days_ahead,manual_offers,exclude_offers,update_cobra)
        scheduled_offers,updated_cobra,switch_notes = sched.schedule_production_drops(sc_dppub_affiliate,days_ahead,manual_offers,exclude_offers,update_cobra=update_cobra)
        print(scheduled_offers)
        all_switch_notes.extend(switch_notes)
        #sched.schedule_ot_drops(sc_dppub_affiliate,days_ahead,manual_offers,exclude_offers,update_cobra=update_cobra) # commented out because we don't need any OT drops
        #sched.schedule_e_drops(sc_dppub_affiliate,days_ahead,manual_offers,exclude_offers,update_cobra=update_cobra) # commented out because we don't need any E drops
        #sched.schedule_mining_drops(sc_dppub_affiliate,days_ahead,exclude_offers=exclude_offers,update_cobra=update_cobra) # commented out because we don't need any mining drops
        
        sched.write_to_log(sc_dppub_affiliate,update_cobra)
        sched.write_affiliate_to_cobra(sc_dppub_affiliate,days_ahead,update_cobra)
        print("\n\n\nCompleted Scheduling for {}.\n\n\n".format(sc_dppub_affiliate))
#         scheduled_offers_array.append(scheduled_offers)
    sched.show_switch_notes(all_switch_notes)
    account_health.account_stats(lexi,to_schedule)
    result = updated_cobra

def schedule_swaps(swap_name,create_swaps=False,override_cpm=False):   
    cobra = cobra_clean.copy()
    log = fvs.load_log()
    sched = scheduling.inputs(fvs=fvs, cs=cs, offs=os, cobra=cobra, cobra_clean=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, emit=emit, offers=offers, lexi=lexi, sc_dppub_affiliate=to_schedule[0], days_ahead=days_ahead, load_log=False, all_aff_log=log)
    sched.schedule_swaps(swap_name,update_cobra=update_cobra,exclude_offers=exclude_offers,create_swaps=create_swaps,override_cpm=override_cpm) 
      
def get_metric_df(metric_cobra):
    predicted_revenues = lexi.groupby(['sc_dppub_affiliate ID','Hitpath Offer ID']).agg({'EMA Revenue':'last'}).rename({'EMA Revenue':'Predicted Revenue'},axis=1)

    metric_df = metric_cobra[(metric_cobra['sc_dppub_affiliate ID'].isin(to_schedule))&(metric_cobra['Date'].dt.date>=date.today())&~(metric_cobra['Hitpath Offer ID'].isna())]
    metric_df = pd.merge(metric_df,predicted_revenues,how='left',on=['Hitpath Offer ID','sc_dppub_affiliate ID'])
    metric_df = metric_df.groupby(['sc_dppub_affiliate ID','Hitpath Offer ID']).agg({'Predicted Revenue':['sum','count']})
    metric_df.columns = ['Total Predicted Revenue','Total Drop Count']
    
    result = metric_df
    

In [2]:
%store -r lexi
%store -r lexi_clean
%store -r lexi_raw

no stored variable or alias lexi
no stored variable or alias lexi_clean
no stored variable or alias lexi_raw


In [3]:
#to_schedule = filepath.user_accounts #list of affiliates to schedule (use schedule_all_affiliates())
#
to_schedule=['HZB_CM.OSR_461452','MBC_CM.OSR_461452','SVT_AMD.PL_461810','HZB_AMD.PL_461810','HZB_AL.PL.3_461838','DSS_TLG.PL_461768','HZB_TLG.PL_461768','MBC_WWM.YFA.2_461680','FLC_WWM.YFA.2_461680'] #the sc_dppub_affiliate for which to schedule (list of strings)
#to_schedule = ['HZB_CM.OSR_461452']
days_ahead=17 #how many days to schedule ahead for
update_cobra=  True #whether to actually write to Cobra or give sample output (True or False)
manual_offers=[] #offers to put to front of queue, still follow gapping rules (list of hitpath ID's) e.g. manual_offers=[1234,5678] 
exclude_offers=[] #offers to ensure we do not schedule (list of hitpath ID's) e.g. exclude_offers=[1234,5678]

cobra, cobra_clean, cobra_sheet, cobra_sheet_df = viper_main.refresh_cobra(secure=False)
emit = viper_main.get_emit()
offers = viper_main.get_offers()
lexi, lexi_clean, lexi_raw = viper_main.get_lexi_fast()

#fvs, cs, os = set_inputs()
fvs = find_viper_settings.inputs(lexi=lexi_clean, cobra=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, offers=offers)    
cs = content_selection.inputs(fvs=fvs, lexi=lexi_clean, cobra=cobra, offers=offers)
os = offer_selection.inputs(fvs=fvs, lexi=lexi_clean, cobra=cobra, offers=offers, emit=emit)
#cobra = fvs.check_for_drop_changes(sc_dppub_affiliate=to_schedule,update_cobra=update_cobra)
start = time.time()

# ** Uncomment one function below to begin **
updated_cobra = schedule_all_affiliates(to_schedule)

swap_name = filepath.user
#schedule_swaps(swap_name,create_swaps=True,override_cpm=False) # we don't need swap drops now in sms 

end = time.time()

print("\nTotal Time to Schedule:",round((end-start),2),"seconds.")



Loading Cobra...


/Users/liliguo/Desktop/SMS-NewData/viper-main/SMS-Viper/infrastructure.py:230: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


Cobra Loaded!

Loading EMIT...
EMIT Loaded!

Loading Offers Sheet...
Offers Sheet Loaded!

Loading Lexi Data...


/Users/liliguo/Desktop/SMS-NewData/viper-main/SMS-Viper/infrastructure.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


Lexi Loading Complete!
Loading Viper Log:
Downloaded local copy in 1.34 seconds, now formatting to dataframe:


/Users/liliguo/Desktop/SMS-NewData/viper-main/SMS-Viper/find_viper_settings.py:971: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  log['Viper Execution Date'] = pd.to_datetime(log['Viper Execution Date'])


Viper Log ready after 1.75 seconds



Beginning Scheduling for HZB_CM.OSR_461452:



**********
Now Scheduling Production Drops:
**********
*** Scheduling Tier One with eCPM ***

*** Completed scheduling tier one; 0 spots to be filled with tier two offers ***

*** Completed scheduling tier two; 0 spots to be filled with tier three offers ***

There are 0 spots that remain empty in the next 17 days.
[]
Completed writing schedule to Cobra!





Completed Scheduling for HZB_CM.OSR_461452.






Beginning Scheduling for MBC_CM.OSR_461452:



**********
Now Scheduling Production Drops:
**********
*** Scheduling Tier One with eCPM ***

*** Completed scheduling tier one; 0 spots to be filled with tier two offers ***

*** Completed scheduling tier two; 0 spots to be filled with tier three offers ***

There are 0 spots that remain empty in the next 17 days.
[]
Completed writing schedule to Cobra!





Completed Scheduling for MBC_CM.OSR_461452.






Beginning Scheduling for SVT_AMD.PL_461810:


In [4]:
""" 
fvs=fvs
cs=cs
offs=os
cobra=cobra
cobra_clean=cobra_clean
cobra_sheet=cobra_sheet
cobra_sheet_df=cobra_sheet_df
emit=emit
offers=offers
lexi=lexi_clean
days_ahead=days_ahead
log = fvs.load_log()
all_aff_log=log
today=date.today()
sc_dppub_affiliate = 'MBC_NPD.RTO_461313'
days_ahead = 10 
update_cobra=False
print_bool=True
metric='eCPM'
"""

" \nfvs=fvs\ncs=cs\noffs=os\ncobra=cobra\ncobra_clean=cobra_clean\ncobra_sheet=cobra_sheet\ncobra_sheet_df=cobra_sheet_df\nemit=emit\noffers=offers\nlexi=lexi_clean\ndays_ahead=days_ahead\nlog = fvs.load_log()\nall_aff_log=log\ntoday=date.today()\nsc_dppub_affiliate = 'MBC_NPD.RTO_461313'\ndays_ahead = 10 \nupdate_cobra=False\nprint_bool=True\nmetric='eCPM'\n"

In [5]:
""" 
        fvs=fvs
        cs=cs
        offs=os
        cobra=cobra
        cobra_clean=cobra_clean
        cobra_sheet=cobra_sheet
        cobra_sheet_df=cobra_sheet_df
        emit=emit
        offers=offers
        lexi=lexi_clean
        days_ahead=days_ahead
        log = fvs.load_log()
        all_aff_log=log
        today=date.today()
        sc_dppub_affiliate = 'HZB_CM.OSR_461452'
        days_ahead = 10 
        update_cobra=False
        print_bool=True
        metric='eCPM'
        offer = '11468'
        segment='A120'
        send_strategy='P'
        manual_offers=[]
        exclude_offers=[]
        write_to_cobra_bool=False
        selection_confidence=''
        given_empties=[]
        print_bool=True
        simple_creative=False
        mining_drops=[8]
        #simple_creative will eliminate the randomness from HTML/CC selection
        sched = scheduling.inputs(fvs=fvs, cs=cs, offs=os, cobra=cobra, cobra_clean=cobra_clean, cobra_sheet=cobra_sheet, cobra_sheet_df=cobra_sheet_df, emit=emit, offers=offers, lexi=lexi_clean, sc_dppub_affiliate=sc_dppub_affiliate, days_ahead=days_ahead, all_aff_log=log)
 
        cobra = sched.cobra
        functions_dict = sched.fvs.find_affiliate_functions(sc_dppub_affiliate)
        offers = sched.offers
        emit = sched.emit
        fvs = sched.fvs
        offs = sched.offs
        ss_offer = sched.ss_offer
        switch_notes = sched.switch_notes
        sc = sc_dppub_affiliate.split('_')[0]
"""    
        

" \n        fvs=fvs\n        cs=cs\n        offs=os\n        cobra=cobra\n        cobra_clean=cobra_clean\n        cobra_sheet=cobra_sheet\n        cobra_sheet_df=cobra_sheet_df\n        emit=emit\n        offers=offers\n        lexi=lexi_clean\n        days_ahead=days_ahead\n        log = fvs.load_log()\n        all_aff_log=log\n        today=date.today()\n        sc_dppub_affiliate = 'HZB_CM.OSR_461452'\n        days_ahead = 10 \n        update_cobra=False\n        print_bool=True\n        metric='eCPM'\n        offer = '11468'\n        segment='A120'\n        send_strategy='P'\n        manual_offers=[]\n        exclude_offers=[]\n        write_to_cobra_bool=False\n        selection_confidence=''\n        given_empties=[]\n        print_bool=True\n        simple_creative=False\n        mining_drops=[8]\n        #simple_creative will eliminate the randomness from HTML/CC selection\n        sched = scheduling.inputs(fvs=fvs, cs=cs, offs=os, cobra=cobra, cobra_clean=cobra_clean, cobra_s

In [6]:
      """ 
        cobra = sched.cobra
        offs = sched.offs
        ss_offer = sched.ss_offer
        functions_dict = sched.fvs.find_affiliate_functions(sc_dppub_affiliate)
        offers = sched.offers
        sc = sc_dppub_affiliate.split('_')[0]

        offer_gap_function = getattr(offs, functions_dict['offer_gap_determination'])
        gap = offer_gap_function(sc_dppub_affiliate,offer)

        empty_slots = sched.find_empty_slots(sc_dppub_affiliate,offer,days_ahead,send_strategy=send_strategy,
                                       manual_offers=manual_offers,exclude_offers=exclude_offers,given_empties=given_empties)
       
        if len(empty_slots) > 0:
            
            next_drop = empty_slots[empty_slots['Date']==empty_slots['Date'].min()].sample(n=1)

            next_drop_date = next_drop['Date'].dt.date.values[0].strftime('%-m/%-d/%Y')

            if sched.check_debt_vertical_gapping(sc_dppub_affiliate,offer,next_drop['Date'].dt.date.values[0]):

    #             if send_strategy=='MI':
    #                 next_drop_number = 8
    #             else:
                next_drop_number = next_drop['Drop Number'].values[0]

#                 campaign = offers[offers['Hitpath Offer ID']==offer]['Scheduling Name'].values[0]
                #campaign = offers.loc[offer]['Scheduling Name']
                try: 
                    campaign = ss_offer.loc[(ss_offer['SS Offers (updated)'].str.contains(offer, na = False)) & (ss_offer['SS Offers (updated)'].str[-4:].str.contains(sc, na = False)), 'SS Offers (updated)' ].values[0]
                except: 
                    print("The offer is not added in the SS.")
                
                next_drop['Offer'] = campaign
                next_drop['Hitpath Offer ID'] = offer
                next_drop['Send Strategy'] = 'P'
                date1 = next_drop['Date'].dt.date.values[0].strftime("%d%b%y")  
                next_drop['Job Name'] = "SS_"+segment[:3] + "_"+segment[4:].replace(".",'-').replace("_",'-')+"_"+ offer +"_"+ send_strategy + "_" +   date1


                cobra.loc[next_drop.index, :] = next_drop[:]
        """

' \n  cobra = sched.cobra\n  offs = sched.offs\n  ss_offer = sched.ss_offer\n  functions_dict = sched.fvs.find_affiliate_functions(sc_dppub_affiliate)\n  offers = sched.offers\n  sc = sc_dppub_affiliate.split(\'_\')[0]\n\n  offer_gap_function = getattr(offs, functions_dict[\'offer_gap_determination\'])\n  gap = offer_gap_function(sc_dppub_affiliate,offer)\n\n  empty_slots = sched.find_empty_slots(sc_dppub_affiliate,offer,days_ahead,send_strategy=send_strategy,\n                                 manual_offers=manual_offers,exclude_offers=exclude_offers,given_empties=given_empties)\n \n  if len(empty_slots) > 0:\n      \n      next_drop = empty_slots[empty_slots[\'Date\']==empty_slots[\'Date\'].min()].sample(n=1)\n\n      next_drop_date = next_drop[\'Date\'].dt.date.values[0].strftime(\'%-m/%-d/%Y\')\n\n      if sched.check_debt_vertical_gapping(sc_dppub_affiliate,offer,next_drop[\'Date\'].dt.date.values[0]):\n\n    #             if send_strategy==\'MI\':\n    #                 next_drop_